# Companion Notebook for Scraping Twitter Using Tweepy

Package Github: https://github.com/tweepy/tweepy

Package Documentation: https://tweepy.readthedocs.io/en/latest/

Article Read-Along: https://towardsdatascience.com/how-to-scrape-more-information-from-tweets-on-twitter-44fd540b8a1f

### Notebook Author: Martin Beck
#### Information current as of August, 13th 2020
<b> Dependencies:</b> Make sure Tweepy is already installed in your Python environment. If not, you can pip install Tweepy to install the package. If you want more information on setting up I have an article [here](https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1) that goes into deeper detail.

## Notebook's Table of Contents<a name="TOC"></a>
<br>
<b>This companion notebook is meant to build on the scraping article and article notebook as it covers more scenarios that may come up and provides more examples.</b>

0. [Credentials and Authorization](#Section0)
<br>Setting up credentials and authorization in order to utilize Tweepy
1. [Getting More Information From Tweets](#Section1)
<br>How to scrape more information from tweets such as favorite count, retweet count, if they're replying to someone else, if turned on the coordinates of where the tweet came from, etc.
2. [Getting User Information From Tweets](#Section2)
<br>How to scrape user information from tweets such as their follower count, total amount of tweets, if they're a verified user, location of where account is registered, etc.
3. [Scraping Tweets With Advanced Queries](#Section3)
<br>How to scrape for tweets using deeper queries such as searching by language of tweets, tweets within a certain location, tweets within specific date ranges, top tweets, etc.
4. [Putting It All Together](#Section4)
<br>Showcasing how you can mix and match the methods shown above to create queries that'll fulfill your data needs.

## Imports for Notebook

In [11]:
# Pip install Tweepy if you don't already have the package
# !pip install tweepy

# Imports
import tweepy
import pandas as pd
import time

## 0. Credentials and Authorization<a name="Section0"></a>
[Return to Table of Contents](#TOC)
<br>Tweepy requires credentials before you can utilize its API. The below code helps setup the notebook for authorization. I already have an an article covering setting up Tweepy and getting credentials [here](https://towardsdatascience.com/how-to-scrape-tweets-from-twitter-59287e20f0f1) if further instructions are needed.

You don't necessarily have to create a credentials file, however if you find youself sharing Tweepy code to other parties I recommend it so you don't accidentally share your credentials. Otherwise skip the below cell and just enter your credentials in and have them hardcoded below.

In [19]:
# Loading in from csv file

credentials_df = pd.read_csv('credentials.csv',header=None,names=['name','key'])

credentials_df

,name,key
0,consumer_key,XXXXXXXXXXX
1,consumer_secret,XXXXXXXXXXX
2,access_token,XXXXXXXXXXX
3,access_secret,XXXXXXXXXXX


In [13]:
# Credentials from csv file

consumer_key = credentials_df.loc[credentials_df['name']=='consumer_key','key'].iloc[0]
consumer_secret = credentials_df.loc[credentials_df['name']=='consumer_secret','key'].iloc[0]
access_token = credentials_df.loc[credentials_df['name']=='access_token','key'].iloc[0]
access_token_secret = credentials_df.loc[credentials_df['name']=='access_secret','key'].iloc[0]

# Credentials hardcoded

# consumer_key = "XXXXX"
# consumer_secret = "XXXXX"
# access_token = "XXXXX"
# access_token_secret = "XXXXXX"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

## 1. Getting More Information From Tweets<a name="Section1"></a>
[Return to Table of Contents](#TOC)
<br>List of information available in tweet object with Tweepy. This is not an exhaustive list but does contain a majority of the available information. If you want an exhaustive list of everything contained in the tweet object there's documentation [here](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview/tweet-object) describing all the attributes. 

String versions of Id's (e.g., id_str, in_reply_to_status_id_str) are used instead to best keep data integrity as there is a possibility for Id's stored as integers to be cut off.

* tweet.user <b>User information is covered in part 2 in greater detail</b><br><br>

* tweet.full_text: <b>Text content of tweet when API is told to pull all contents of tweets that have more than 140 characters</b><br><br>

* tweet.text: Text content of tweet
* tweet.created_at: Date tweet was created
* tweet.id_str: Id of tweet
* tweet.user.screen_name: Username of tweet's author
* tweet.coordinates: Geographic location as reported by user or client. May be null that is why extract_coordinates function below was created
* tweet.place: Indicates place associated with tweet where user signed up with like Las Vegas, NV. May be null that so extract_place function below was created
* tweet.retweet_count: Count of retweets
* tweet.favorite_count: Count of favorites
* tweet.lang: Indicates a BCP 47 language identifier corresponding to machine detected language of tweet text.
* tweet.source: Source where tweet was posted through. Ex: Twitter Web Client
* tweet.in_reply_to_status_id_str: If a tweet is a reply, the original tweet's id. Can be null if tweet is not a reply
* tweet.in_reply_to_user_id_str: If a tweet is a reply, string representation of original tweet's user id
* tweet.is_quote_status: If tweet is a quote tweet

### Query by Username
I created three functions to build off of based off of various scenarios that are likely to happen for someone scraping tweets from users. After each function I call them to showcase an example of them being used.

#### F0. extract_coordinates and extract_place
These functions check for if a tweet has either coordinate information or place information and extract the pertinent information from their json. These are separate functions because they can be nullable so it's important to check first if they have them then to extract and replace in the dataframe.

#### F1. scrape_user_tweets
This function scrapes a single users tweets and exports the data as a csv or excel file

#### F2. scrape_multiple_users_multifile
This function scrapes multiple users based on a list and exports separate csv or excel files per user.

#### F3. scrape_multiple_users_singlefile
This function scrapes multiple users based on a list and exports one csv or excel file containing all tweets

In [14]:
# Function created to extract coordinates from tweet if it has coordinate info
# Tweets tend to have null so important to run check
# Make sure to run this cell as it is used in a lot of different functions below
def extract_coordinates(row):
    if row['Tweet Coordinates']:
        return row['Tweet Coordinates']['coordinates']
    else:
        return None

# Function created to extract place such as city, state or country from tweet if it has place info
# Tweets tend to have null so important to run check
# Make sure to run this cell as it is used in a lot of different functions below
def extract_place(row):
    if row['Place Info']:
        return row['Place Info'].full_name
    else:
        return None

In [127]:
def scrape_user_tweets(username, max_tweets):
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.user_timeline,id=username).items(max_tweets)

    # List comprehension pulling chosen tweet information from tweets iterable object
    # Add or remove tweet information you want in the below list comprehension
    tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.coordinates,
                   tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang,
                   tweet.source, tweet.in_reply_to_status_id_str, 
                    tweet.in_reply_to_user_id_str, tweet.is_quote_status,
                    ] for tweet in tweets]

    # Creation of dataframe from tweets_list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name', 'Tweet Coordinates', 'Place Info',
                                                 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id',
                                                  'Replied Tweet User Id Str', 'Quote Status Bool'])
    
    # Checks if there are coordinates attached to tweets, if so extracts them
    tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
    
    # Checks if there is place information available, if so extracts them
    tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)
    
    # Uncomment/comment below lines to decide between creating csv or excel file 
    tweets_df.to_csv('{}-tweets.csv'.format(username), sep=',', index = False)
#     tweets_df.to_excel('{}-tweets.xlsx'.format(username), index = False)

In [128]:
# Creating example username to scrape from
username = 'random'

# Max recent tweets pulls x amount of most recent tweets from that user
max_tweets = 150

# Function will scrape username, attempt to pull max_tweet amount, and create csv/excel file from data.
scrape_user_tweets(username,max_tweets)

In [131]:
def scrape_multiple_users_multifile(username_list, max_tweets_per):
    # Looping through each username in user list
    
    for username in username_list:   
        # Creation of query method using parameters
        tweets = tweepy.Cursor(api.user_timeline,id=username).items(max_tweets_per)

        # List comprehension pulling chosen tweet information from tweets iterable object
        # Add or remove tweet information you want in the below list comprehension
        tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.coordinates,
                   tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang,
                   tweet.source, tweet.in_reply_to_status_id_str, 
                    tweet.in_reply_to_user_id_str, tweet.is_quote_status,] for tweet in tweets]

        # Creation of dataframe from tweets_list
        # Add or remove columns as you remove tweet information
        tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name', 'Tweet Coordinates', 'Place Info',
                                                 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id',
                                                  'Replied Tweet User Id Str', 'Quote Status Bool'])
        
        # Checks if there are coordinates attached to tweets, if so extracts them
        tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
        
        # Checks if there is place information available, if so extracts them
        tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)
        
        # Uncomment/comment below lines to decide between creating csv or excel file 
        tweets_df.to_csv('{}-tweets.csv'.format(username), sep=',', index = False)
#         tweets_df.to_excel('{}-tweets.xlsx'.format(username), index = False)

In [130]:
# Creating example user list with 3 users
user_name_list = ['jack','billgates','random']

# Max recent tweets pulls x amount of most recent tweets from that user
max_tweets_per = 150

# Function will scrape each user, attempting to pull max_tweet amount, and create csv/excel file per user.
scrape_multiple_users_multifile(user_name_list, max_tweets_per)

In [134]:
def scrape_multiple_users_singlefile(username_list, max_tweets_per):
    # Creating master list to contain all tweets
    master_tweets_list = []
    
    # Looping through each username in user list
    for username in user_name_list:
        # Creation of query method using parameters
        tweets = tweepy.Cursor(api.user_timeline,id=username).items(max_tweets_per)
        
        # List comprehension pulling chosen tweet information from tweets iterable object
        # Appending new tweets per user into the master tweet list
        # Add or remove tweet information you want in the below list comprehension
        for tweet in tweets:
            master_tweets_list.append((tweet.text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.coordinates,
                   tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang,
                   tweet.source, tweet.in_reply_to_status_id_str, 
                    tweet.in_reply_to_user_id_str, tweet.is_quote_status))
        
    # Creation of dataframe from tweets_list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(master_tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name', 'Tweet Coordinates', 'Place Info',
                                                 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id',
                                                  'Replied Tweet User Id Str', 'Quote Status Bool'])

    # Checks if there are coordinates attached to tweets, if so extracts them
    tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
    
    # Checks if there is place information available, if so extracts them
    tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)
    
    # Uncomment/comment below lines to decide between creating csv or excel file 
    tweets_df.to_csv('multi-user-tweets.csv', sep=',', index = False)
#     tweets_df.to_excel('multi-user-tweets.xlsx', index = False)

In [133]:
# Creating example user list with 3 users
user_name_list = ['jack','billgates','random']

# Max recent tweets pulls x amount of most recent tweets from that user
max_tweets_per = 150

# Function will scrape each user, attempting to pull max_tweet amount, and create one csv/excel file containing all data name multi-user-tweets.
scrape_multiple_users_singlefile(user_name_list, max_tweets_per)

## Allowing API to Access up to 280 Characters From Tweets

In the cursor parameters add tweet_mode='extended' to access tweet text that goes beyond Twitter's original 140 character limit.

If tweet_mode is set to extended the tweet attribute tweet.text becomes tweet.full_text isntead.

In [17]:
def scrape_extended_tweets(username, max_tweets):
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.user_timeline,id=username, tweet_mode='extended').items(max_tweets)

    # List comprehension pulling chosen tweet information from tweets iterable object
    # Add or remove tweet information you want in the below list comprehension
    tweets_list = [[tweet.full_text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.coordinates,
                   tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang,
                   tweet.source, tweet.in_reply_to_status_id_str, 
                    tweet.in_reply_to_user_id_str, tweet.is_quote_status,
                    ] for tweet in tweets]

    # Creation of dataframe from tweets_list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name', 'Tweet Coordinates', 'Place Info',
                                                 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id',
                                                  'Replied Tweet User Id Str', 'Quote Status Bool'])
    
    # Checks if there are coordinates attached to tweets, if so extracts them
    tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
    
    # Checks if there is place information available, if so extracts them
    tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)
    
    # Uncomment/comment below lines to decide between creating csv or excel file 
    tweets_df.to_csv('{}-tweets.csv'.format(username), sep=',', index = False)
#     tweets_df.to_excel('{}-tweets.xlsx'.format(username), index = False)

In [18]:
# Creating example username to scrape from
username = 'billgates'

# Max recent tweets pulls x amount of most recent tweets from that user
max_tweets = 150

# Function will scrape username, attempt to pull max_tweet amount, and create csv/excel file from data.
scrape_extended_tweets(username,max_tweets)

### Query by Text Search
I created one function to build off of for scraping tweets by text search.

#### F1. scrape_text_query
This function scrapes tweets from Twitter based on the text search and exports the data as a csv or excel file

In [9]:
def scrape_text_query(text_query, max_tweets):
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.search,q=text_query, tweet_mode='extended').items(max_tweets)

    # List comprehension pulling chosen tweet information from tweets iterable object
    # Add or remove tweet information you want in the below list comprehension
    tweets_list = [[tweet.full_text, tweet.created_at, tweet.id_str, tweet.user.screen_name, tweet.coordinates,
               tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang,
               tweet.source, tweet.in_reply_to_status_id_str, 
                tweet.in_reply_to_user_id_str, tweet.is_quote_status,
                ] for tweet in tweets]

    # Creation of dataframe from tweets_list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name', 'Tweet Coordinates', 'Place Info',
                                                 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id',
                                                  'Replied Tweet User Id Str', 'Quote Status Bool'])

    # Checks if there are coordinates attached to tweets, if so extracts them
    tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
    
    # Checks if there is place information available, if so extracts them
    tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)

    # Uncomment/comment below lines to decide between creating csv or excel file 
    tweets_df.to_csv('{}-tweets.csv'.format(text_query), sep=',', index = False)
#     tweets_df.to_excel('{}-tweets.xlsx'.format(text_query), index = False)

In [10]:
# Input search query to scrape tweets and name csv file
text_query = 'Coronavirus'

# Max recent tweets pulls x amount of most recent tweets from that user
max_tweets = 150

# Function scrapes for tweets containing text_query, attempting to pull max_tweet amount and create csv/excel file containing data.
scrape_text_query(text_query, max_tweets)

## 2. Getting User Information From Tweets<a name="Section2"></a>
[Return to Table of Contents](#TOC)

<b>Tweepy excels in this category. Having more access to user information than GetOldTweets3.</b>
<br>List of information available in user object with Tweepy. This is not an exhaustive list but does contain a majority of the available information. If you want an exhaustive list of everything contained in the tweet object there's documentation [here](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/overview/user-object) describing all the attributes. 

String versions of Id's (e.g., id_str, user.id_str) are used instead to best keep data integrity as there is a possibility for Id's stored as integers to be cut off.

* tweet.text: Text content of tweet
* tweet.created_at: Date tweet was created
* tweet.id_str: Id of tweet
* tweet.user.name: Name of the user as they've defined it
* tweet.user.screen_name: Username of tweet's author, commonly called User @ name
* tweet.user.id_str: Use id of tweet's author
* tweet.user.location: User defined location for account's profile. Can be nullable
* tweet.user.url: URL provided by user in bio. Can be nullable
* tweet.user.description: Text in user bio. Can be nullable
* tweet.user.verified: Boolean indicating whether user has a verified account
* tweet.user.followers_count: Count of followers user has
* tweet.user.friends_count: Count of other users that user is following
* tweet.user.favourites_count: Count of tweets user has liked in the account's lifetime
* tweet.user.statuses_count: Count of tweets (including retweets) issued by user
* tweet.user.listed_count: Count of public lists that user is member of
* tweet.user.created_at: Date that the user account was created on Twitter
* tweet.user.profile_image_url_https: HTTPS-based URL pointing to user's profile image
* tweet.user.default_profile: When true, indicates user has not altered the theme or background of user profile
* tweet.user.default_profile_image: When true, indicates if user has not uploaded their own profile image and default image is used instead

### Query by Text Search
I created one function to build off of that searches by text and pulls all user information available.

#### F1. scrape_user_information
This function scrapes tweets from Twitter based on the text search, pulls user information and exports the data as a csv or excel file

In [137]:
def scrape_user_information(text_query, max_tweets):
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.search,q=text_query).items(max_tweets)

    # List comprehension pulling chosen tweet information from tweets iterable object
    # Add or remove tweet information you want in the below list comprehension
    tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.user.name, tweet.user.screen_name, 
                    tweet.user.id_str, tweet.user.location, tweet.user.url,
                    tweet.user.description, tweet.user.verified, tweet.user.followers_count,
                    tweet.user.friends_count, tweet.user.favourites_count, tweet.user.statuses_count,
                    tweet.user.listed_count, tweet.user.created_at, tweet.user.profile_image_url_https,
                    tweet.user.default_profile, tweet.user.default_profile_image] for tweet in tweets]

    # Creation of dataframe from tweets_list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter Username', 'Twitter @ name',
                                                 'Twitter User Id', 'Twitter User Location', 'URL in Bio', 'Twitter Bio',
                                                 'User Verified Status', 'Users Following Count',
                                                 'Number users this account is following', 'Users Number of Likes', 'Users Tweet Count',
                                                 'Lists Containing User', 'Account Created Time', 'Profile Image URL',
                                                 'User Default Profile', 'User Default Profile Image'])

    # Uncomment/comment below lines to decide between creating csv or excel file 
    tweets_df.to_csv('{}-userinfo-tweets.csv'.format(text_query), sep=',', index = False)
    # tweets_df.to_excel('{}-userinfo-tweets.xlsx'.format(text_query), index = False)

In [138]:
# Input search query to scrape tweets and name csv file
text_query = 'Coronavirus'

# Max recent tweets pulls x amount of most recent tweets from that user
max_tweets = 150

# Function scrapes for tweets containing text_query, attempting to pull max_tweet amount and create csv/excel file containing data.
scrape_user_information(text_query, max_tweets)

## 3. Scraping Tweets With Advanced Queries<a name="Section3"></a>
[Return to Table of Contents](#TOC)
<br>List of query methods available with Tweepy. This is not an exhaustive list but does contain a majority of the methods available. If you want an exhaustive list of everything available there's documentation [here](https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets).

* q = str: Setting query based on text
* geocode = str "lat,long,radius": Setting location of query and radius
* lang = str: Setting language of query, full list of language codes [here](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes)</b>
* result_type = str "mixed"/"recent"/"popular": Setting popularity preference of query
* until = str "yyyy-mm-dd": Setting upper bound date on query, if using standard search API be cognizant of 7-day limit
* since_id = str or int: Returns results with Id's more recent than given Id
* max_id = str or int: Returns results with Id's older than given Id
* count = int: Number of tweets to return per page. Max is 100, defaults to 15

I created a function to build off of based that utilize the different query methods available with Tweepy. As you can see you can mix and match the above methods in any way. It's important to remember that the more restrictive you make the search the more likely that a smaller amount of tweets that will come up.

#### F1. scrape_advanced_queries
This function queries by using geocode to set a location to query for tweets and restrict within a certain radius, lang to scrape for tweets written in a specific language, result_type to search for tweets based on popularity, until to set an upper bound date on tweets, since_id to set a restriction on the oldest id possible, and max_id to set a restriction on the earliest id possible

In [139]:
def scrape_advanced_queries(coordinates, language, result_type, until_date, max_tweets):
    # Creation of query method using parameters
    tweets = tweepy.Cursor(api.search, geocode=coordinates, lang=language, result_type = result_type, 
                           until = until_date, count = 100).items(max_tweets)

    # List comprehension pulling chosen tweet information from tweets iterable object
    # Add or remove tweet information you want in the below list comprehension
    tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.favorite_count, tweet.user.screen_name, 
                    tweet.user.id_str, tweet.user.location, tweet.user.url, 
                    tweet.user.verified, tweet.user.followers_count,
                    tweet.user.friends_count, tweet.user.statuses_count,
                    tweet.user.default_profile_image, tweet.lang] for tweet in tweets]

    # Creation of dataframe from tweets_list
    # Add or remove columns as you remove tweet information
    tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Tweet Favorite Count', 'Twitter @ name',
                                                 'Twitter User Id', 'Twitter User Location', 'URL in Bio','User Verified Status', 'Users Current Following Count',
                                                 'Number of accounts user is following', 'Users Tweet Count',
                                                 'Profile Image URL','Tweet Language'])
    
    # Uncomment/comment below lines to decide between creating csv or excel file 
    tweets_df.to_csv('advancedqueries-tweets.csv', sep=',', index = False)
    # tweets_df.to_excel('advancedqueries-tweets.xlsx', index = False)

In [140]:
# Example may no longer show tweets if until_date falls outside 
# of 7-day period from when you run cell

coordinates = '19.402833,-99.141051,50mi'
language = 'es'
result_type = 'recent'
until_date = '2020-08-10'
max_tweets = 150

scrape_advanced_queries(coordinates, language, result_type, until_date, max_tweets)

## 4. Putting It All Together<a name="Section4"></a>
[Return to Table of Contents](#TOC)
<br>
Great, we now know how to pull more information from tweets and querying with advanced parameters. The great thing is how easy it is to mix and match whatever you want to search for. While it was shown above several times. The point is that you can mix and match the information you want from the tweets and the type of queries you conduct. It's just important that you update the column names in the pandas dataframe so you don't get errors.

<br>
Below is an example of a search for 150 tweets with 'Coronavirus' in it that occurred within a 50 mile radius of Las Vegas, NV. Which in this case has the geo coordinates of lat 36.169786, long -115.139858


In [142]:
text_query = 'Coronavirus'
coordinates = '36.169786,-115.139858,50mi'
max_tweets = 150

# Creation of query method using parameters
tweets = tweepy.Cursor(api.search, q = text_query, geocode = coordinates, count = 100).items(max_tweets)

# List comprehension pulling chosen tweet information from tweets iterable object
# Add or remove tweet information you want in the below list comprehension
tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.favorite_count, tweet.user.screen_name, 
                tweet.user.id_str, tweet.user.location, tweet.user.followers_count, tweet.coordinates, tweet.place] for tweet in tweets]

# Creation of dataframe from tweets_list
# Add or remove columns as you remove tweet information
tweets_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Tweet Favorite Count', 'Twitter @ name',
                                             'Twitter User Id', 'Twitter User Location', 'Users Current Following Count', 'Tweet Coordinates', 'Place Info'])

# Checks if there are coordinates attached to tweets, if so extracts them
tweets_df['Tweet Coordinates'] = tweets_df.apply(extract_coordinates,axis=1)
    
# Checks if there is place information available, if so extracts them
tweets_df['Place Info'] = tweets_df.apply(extract_place,axis=1)

# Uncomment/comment below lines to decide between creating csv or excel file 
tweets_df.to_csv('put-together-tweets.csv', sep=',', index = False)
# tweets_df.to_excel('put-together-tweets.xlsx', index = False)